# Preloaded code

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def count_metrics(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return mae, mse, rmse, r2 


SEED = 42


data = pd.read_csv('flights.csv', index_col=0)


y = data['DEPARTURE_DELAY']
data = data.drop('DEPARTURE_DELAY', axis=1)


var_cat = ['MONTH', 'DAY_OF_WEEK', 'AIRLINE']
var_num = sorted(list(set(data.columns) - set(var_cat)))


data = pd.get_dummies(data, drop_first=True, columns=var_cat)


X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=SEED)


scaler = StandardScaler()
scaler.fit(X_train[var_num])
X_train[var_num] = scaler.transform(X_train[var_num])
X_test[var_num] = scaler.transform(X_test[var_num])

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

# Урок 1. Кросс-валидация

### Задание - Валидация модели
- Импортируйте функцию cross_val_score из модуля sklearn.model_selection
- Импортируйте функцию LinearRegression из модуля sklearn.linear_model
- Объявите модель, надкоторой будете работат
- Оцените кросс-валидацией метрику качества. Параметрами функции cross_val_score будет заданная модель, тренировочная выборка, количество фолдов для кросс-валидации, а также метрика качества в виде текста, присвоенная параметру scoring

In [27]:
# Импортируйте функцию cross_val_score
from sklearn.model_selection import cross_val_score
# Импортируйте функцию LinearRegression
from sklearn.linear_model import LinearRegression
# Объявите модель
model = LinearRegression()
# Вызовите функцию cross_val_score
mse_val = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

### Задание
- Импортируйте библиотеку numpy
- Найдите средний RMSE кросс-валидации:
    - домножьте mse_val на -1
    - функцией np.sqrt возьмите корень
    - найдите среднее значение ошибок функцией np.mean

In [28]:
# Импортируйте библиотеку numpy
import numpy as np
# Найдите средний RMSE кросс-валидации
rmse_val = (np.mean(np.sqrt(-1*mse_val)))
print(rmse_val)

42.604683020728274


# Урок 2. Подбор параметров по решетке

### Задание - Подбор параметра - подготовка

- Импортируйте функцию Lasso из модуля sklearn.linear_model
- Задайте параметр альфа
- Объявите модель 

In [17]:
# Импортируйте функцию Lasso
from sklearn.linear_model import Lasso

# Задайте параметры
params = {'alpha': [0.001, 0.01, 0.05, 0.1, 0.5, 0.8, 1, 5, 10]}

# Объявите модель
model_lasso = Lasso() 

### Задание - обучение решетки
- Импортируйте функцию GridSearchCV из модуля sklearn.model_selection
- Объявите поиск по решетке GridSearchCV()
    - первый параметр - модель model_lasso
    - второй параметр - словарь с параметрами params
    - третий параметр - стратегия кросс-валидации
    - четвертый параметр - метрика, по которой оптмизируется алгоритм
    
- Обучите решетку методов fit()

In [18]:
# Импортируйте функцию GridSearchCV
from sklearn.model_selection import GridSearchCV

# Объявите поиск по решетке
grid_cv_lasso = GridSearchCV(model_lasso, params, cv=5, scoring='neg_mean_squared_error')

# Обучите решетку
grid_cv_lasso.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [0.001, 0.01, 0.05, 0.1, 0.5, 0.8, 1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

### Задание - поиск лучших параметров
- Выведите на печать лучшее значение параметра, оно записано в поле best_params_ объекта решетки
- Выведите на печать лучшее значение метрики качества, оно записано в поле best_score_ объекта решетки, домноженное на -1


In [19]:
# Распечатайте лучшее значение параметра
print('Лучшее значение параметра:', grid_cv_lasso.best_params_)
# Распечатайте лучшее значение метрики
print('Лучшее качество:', -1*grid_cv_lasso.best_score_)

Лучшее значение параметра: {'alpha': 0.01}
Лучшее качество: 1816.6415245232438


### Задание - сохранить и применить модель
- Запишите наилучшую модель в переменную model_lasso. Она сохранена в поле best_estimator_ объекта решетки
- Сделайте прогнозы на тестовой выборке методом predict()
- Посчитайте метрики качества пользовательской функцией count_metrics

In [25]:
# Запишите наилучшую модель 
model_lasso = grid_cv_lasso.best_estimator_

# Сделайте прогнозы на тестовой выборке
y_pred_lasso = model_lasso.predict(X_test)

# Посчитайте метрики качества
mae_lasso, mse_lasso, rmse_lasso, r2_lasso = count_metrics(y_test, y_pred_lasso)

print("MAE:", mae_lasso)
print('MSE:', mse_lasso)
print('RMSE', rmse_lasso)
print('R2', r2_lasso)

MAE: 23.238005418038664
MSE: 1828.871028581459
RMSE 42.76530168935394
R2 0.12674176583103247


### Задание - подбор параметра для случайного леса

- Импортируйте функцию RandomForestRegressor из модуля sklearn.ensemble 
- Задайте параметры случайного леса, которые будут тестироваться:
    - количество деревьев 'n_estimators': [10, 50, 100]
    - глубина деревьев 'max_depth': [5, 10, 15]
- Объявите модель RandomForestRegressor и запишите в переменную model_rfr
- Объявите поиск по решетке GridSearchCV()
    - первый параметр - модель model_rfr
    - второй параметр - словарь с параметрами params_forest
    - третий параметр - стратегия кросс-валидации
    - четвертый параметр - метрика, по которой оптмизируется алгоритм

In [32]:
# Импортируйте функцию RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

# Задайте параметры
params_forest = {'n_estimators': [10, 50, 100], 'max_depth': [5, 10, 15]}

# Объявите модель
model_rfr = RandomForestRegressor()

# Объявите решетку
grid_cv_rf = GridSearchCV(model_rfr, params_forest, cv=5, scoring='neg_mean_squared_error')

### Задание - обучение и лучшие параметры модели
- Обучите решетку методов fit()
- Выведите на печать лучшее значение параметра, оно записано в поле best_params_ объекта решетки
- Выведите на печать лучшее значение метрики качества, оно записано в поле best_score_ объекта решетки, домноженное на -1

In [33]:
# Обучите решетку
grid_cv_rf.fit(X_train, y_train)

# Распечатайте лучшее значение параметра
print('Лучшее значение параметра:', grid_cv_rf.best_params_)
# Распечатайте лучшее значение метрики
print('Лучшее качество:', -1*grid_cv_rf.best_score_)

Лучшее значение параметра: {'max_depth': 15, 'n_estimators': 100}
Лучшее качество: -98.04336494974584


### Задача - Прогноз по лучшей комбинации 
- Запишите наилучшую модель в переменную model_rfrЁ. Она сохранена в поле best_estimator_ объекта решетки
- Сделайте прогнозы на тестовой выборке методом predict()
- Посчитайте метрики качества пользовательской функцией count_metrics

In [34]:
# Запишите наилучшую модель 
model_rfr = grid_cv_rf.best_estimator_

# Сделайте прогнозы на тестовой выборке
y_pred_rfr = model_rfr.predict(X_test)

# Посчитайте метрики качества
mae_rfr, mse_rfr, rmse_rfr, r2_rfr = count_metrics(y_test, y_pred_rfr)

print("MAE:", mae_rfr)
print('MSE:', mse_rfr)
print('RMSE', rmse_rfr)
print('R2', r2_rfr)

MAE: 2.7552037892536476
MSE: 29.50298764004235
RMSE 5.431665273195906
R2 0.985912769962114


# Урок 3. Сравнение валидированных моделей

### Задание
- Создайте лист с названиями моделей ['Линейная регрессия', 'Lasso-регрессия', 'Регрессия случайным лесом']
- Создайте лист с метрикой RMSE для соответствующих моделей [rmse_val, rmse_lasso, rmse_rfr] 
- Создайте датафрейм metrics_df из словаря metrics_dict функцией pd.DataFrame()

In [36]:
# Создайте лист с названиями моделей
models = ['Линейная регрессия', 'Lasso-регрессия', 'Регрессия случайным лесом']
# Создайте лист с метрикой MSE для соответствующих моделей
rmse = [rmse_val, rmse_lasso, rmse_rfr] 
# Соберите в словарь модели и метрики
metrics_dict = {'Модели': models,
               'RMSE': rmse}
# Соберите датафрейм из словаря metrics_dict
metrics_df = pd.DataFrame(metrics_dict)
metrics_df

,Модели,RMSE
0,Линейная регрессия,42.604683
1,Lasso-регрессия,42.765302
2,Регрессия случайным лесом,5.431665


### Закрыть инструкцией по работе над моделью
от и до